<a href="https://colab.research.google.com/github/Montelongo1992/Avance2-Equipo3/blob/main/Avance2_Equipo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Construcción de Características.**

Cargar y Preprocesar Imágenes: Se cargaron imágenes procesadas de personas autorizadas y no autorizadas, convirtiéndolas a escala de grises y redimensionándolas a 128x128 píxeles. Esto aseguró la homogeneidad en el tamaño de las imágenes para facilitar el procesamiento posterior.
Aplanamiento de Imágenes: Las imágenes fueron aplanadas, convirtiendo la representación bidimensional (128x128 píxeles) en vectores unidimensionales para facilitar el uso de algoritmos de aprendizaje automático.

**Justificacion**

Preprocesamiento aplicado se fundamenta en la necesidad de optimizar los datos para su uso en algoritmos de aprendizaje automático. La conversión de las imágenes a escala de grises elimina la información de color irrelevante, reduciendo la complejidad y posibles ruidos en los datos. El redimensionamiento a 128x128 píxeles asegura que todas las imágenes tengan un formato uniforme, lo que facilita su procesamiento y reduce los costos computacionales. El aplanamiento de las imágenes transforma las matrices bidimensionales en vectores unidimensionales, lo que permite que cada píxel sea tratado como una característica individual por los modelos de clasificación, garantizando así la compatibilidad y eficacia en el análisis de patrones. Estos pasos contribuyen de manera significativa a mejorar el rendimiento del modelo y asegurar su eficiencia durante el entrenamiento.

In [68]:
# Montar Google Drive para jalar imagenes preprocesadas.
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
from collections import Counter

# Directorios donde están almacenadas las imágenes de personas autorizadas y no autorizadas ya procesadas
carpeta_autorizados = "/content/drive/MyDrive/data-imagenes/imagenes_procesadas_autorizadas"
carpeta_no_autorizados = "/content/drive/MyDrive/data-imagenes/imagenes_procesadas_no_autorizadas"

# Función para cargar las imágenes de las carpetas correspondientes
def cargar_imagenes_a_matriz(carpeta, etiqueta_clase, tamano_imagen=(128, 128)):
    imagenes = []
    etiquetas = []

    if os.path.exists(carpeta) and len(os.listdir(carpeta)) > 0:
        for archivo in os.listdir(carpeta):
            img = cv2.imread(os.path.join(carpeta, archivo), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                # Redimensionar a 128x128 píxeles
                img = cv2.resize(img, tamano_imagen)
                imagenes.append(img)

                # Etiquetamos las imágenes en base al valor pasado por parámetro (1 para autorizados, 0 para no autorizados)
                etiquetas.append(etiqueta_clase)
        # Convertimos a matrices
        imagenes = np.array(imagenes)
        etiquetas = np.array(etiquetas)

        # Aplanamos las imágenes para tener una representación vectorial
        imagenes_aplanadas = imagenes.reshape(imagenes.shape[0], -1)
    else:
        print(f"Error: No se encontraron imágenes en {carpeta}")
        imagenes_aplanadas, etiquetas = np.array([]), np.array([])

    return imagenes_aplanadas, etiquetas

# Cargar imágenes desde las rutas con etiquetas correctas
X_autorizados, y_autorizados = cargar_imagenes_a_matriz(carpeta_autorizados, etiqueta_clase=1)
X_no_autorizados, y_no_autorizados = cargar_imagenes_a_matriz(carpeta_no_autorizados, etiqueta_clase=0)

# Verificar si ambas clases de imágenes fueron cargadas correctamente
if X_autorizados.size == 0 or X_no_autorizados.size == 0:
    print("Error: Asegúrate de que las imágenes estén en las carpetas correctas.")
else:
    # Combinar los datos de ambas clases
    X = np.concatenate((X_autorizados, X_no_autorizados), axis=0)
    y = np.concatenate((y_autorizados, y_no_autorizados), axis=0)

    # Mostrar la forma de los datos cargados
    print(f"Forma de las matrices de las imágenes: {X.shape}")
    print(f"Etiquetas: {y.shape}")

    # Verificar distribución de clases
    print(f"Distribución de clases antes de la división: {Counter(y)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Forma de las matrices de las imágenes: (1144, 16384)
Etiquetas: (1144,)
Distribución de clases antes de la división: Counter({1: 572, 0: 572})


**2. Normalización**

Se utilizó StandardScaler para normalizar los datos de entrada antes de aplicar PCA. Esto asegura que todas las características (valores de píxeles) tengan una media apropiada.

**Justificación:**

La normalización es esencial porque los modelos de aprendizaje automático, especialmente aquellos basados en distancias como SVM o regresión logística, son sensibles a las escalas de los datos. Al normalizar los datos, garantizamos que las características tengan el mismo impacto en el proceso de entrenamiento.

In [69]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Normalizar los datos
escalador = StandardScaler()
X_normalizado = escalador.fit_transform(X)



**3. Selección / Extracción de Características.**

Se aplicó PCA para reducir la dimensionalidad de los datos, pasando de 16,384 características (píxeles aplanados) a solo 50 componentes principales. Esto mejora la velocidad de entrenamiento y reduce la complejidad del modelo.
Justificación cuantitativa: El uso de PCA redujo drásticamente la cantidad de características mientras se mantenía la mayor parte de la información relevante, como se puede ver en la retención de la varianza.

**Justificación:**

La selección de características con PCA se alineó a los objetivos del análisis de reducir la dimensionalidad sin perder información relevante. Esto facilita la interpretación y reduce el riesgo de sobreajuste, además de mejorar la eficiencia computacional.
La reducción de dimensionalidad es clave para evitar un modelo complejo con demasiadas características que puedan inducir ruido en el proceso de clasificación.

In [70]:
# Aplicar PCA
pca = PCA(n_components=50)  # Probar con 50 componentes
X_pca = pca.fit_transform(X_normalizado)
print(f"Forma después de PCA con 50 componentes: {X_pca.shape}")


# Mostrar la nueva forma de los datos después de PCA
print(f"Forma después de PCA: {X_pca.shape}")

Forma después de PCA con 50 componentes: (1144, 50)
Forma después de PCA: (1144, 50)


**4. Conclusiones.**

Se incluyó una conclusión clara sobre la preparación de los datos en el contexto de la metodología CRISP-ML. La fase de preparación ha consistido en normalizar, aplicar PCA para extraer características relevantes y ajustar el modelo de manera que la complejidad de los datos se reduzca sin sacrificar precisión.

**Conclusión general:**
En el contexto de CRISP-ML, la fase de preparación de datos fue realizada cumpliendo con los objetivos de limpieza, reducción de dimensionalidad y selección de características. El conjunto de datos está ahora en una forma óptima para su uso en modelos de aprendizaje automático, lo que nos permite avanzar hacia las fases de modelado y evaluación con confianza en que los datos han sido correctamente procesados.